In [1]:
%%sh
pwd

/workspace/School/Deep_Neural_Nets/Final_Project/Working_Branches/aaron_working_2024-04-26/src


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate, Flatten, Conv2D, MaxPooling2D, AvgPool2D, Lambda
from keras.optimizers import Adam
from keras import Model
import math
import tensorflow as tf
from keras.models import Sequential

2024-04-27 16:41:26.443025: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dfY = pd.read_csv("/workspace/School/Deep_Neural_Nets/Final_Project/all_data/train_labels.csv")[:1500]
dfY["path"] = [f"/workspace/School/Deep_Neural_Nets/Final_Project/all_data/images/{id}/{i:03d}.png" for id, i in zip(dfY["chain_id"], dfY["i"])]
dfY["base_path"] = [f"/workspace/School/Deep_Neural_Nets/Final_Project/all_data/images/{id}/000.png" for id, i in zip(dfY["chain_id"], dfY["i"])]
x_train, x_test, y_train, y_test = train_test_split(dfY[["path", "base_path"]], dfY[["x", "y", "z", "qw","qx","qy","qz"]], test_size=0.25, random_state=42)
x_train = pd.DataFrame(x_train, columns=["path", "base_path"])
x_test = pd.DataFrame(x_test, columns=["path", "base_path"])

In [4]:
x_test.head()


,path,base_path
1116,/workspace/School/Deep_Neural_Nets/Final_Proje...,/workspace/School/Deep_Neural_Nets/Final_Proje...
1368,/workspace/School/Deep_Neural_Nets/Final_Proje...,/workspace/School/Deep_Neural_Nets/Final_Proje...
422,/workspace/School/Deep_Neural_Nets/Final_Proje...,/workspace/School/Deep_Neural_Nets/Final_Proje...
413,/workspace/School/Deep_Neural_Nets/Final_Proje...,/workspace/School/Deep_Neural_Nets/Final_Proje...
451,/workspace/School/Deep_Neural_Nets/Final_Proje...,/workspace/School/Deep_Neural_Nets/Final_Proje...


In [9]:
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv')

# Loads all images found and trains given model
def load_images_and_train_model(model, is_pos_model):
    with tf.device('/GPU:0'):
        model.summary()
        
        #for i in range(batches):
            #batch = load_new_batch()
            #model.fit([np.asarray(batch[0]), np.asarray(batch[1])], y_train[100 * i : 100*(i+1)], epochs=40, batch_size=batch_size_b)
        if is_pos_model:
            ypos = pd.DataFrame(y_train, index=y_train.index, columns=["x", "y", "z"])
            model.fit(CustomDataGenerator(x_train, y_train, batch_size=100), epochs=3)
        else:
            yquat = pd.DataFrame(y_train, index=y_train.index, columns=["qw", "qx", "qy", "qz"])
            model.fit(CustomDataGenerator(x_train, yquat, batch_size=100), epochs=3)


        
    #     ypreddf = pd.DataFrame(ypred.T, index=image_label_ds.take(count * 100 * 0.2).index_array, columns=["x", "y", "z"])
    #     dfY = pd.read_csv("../data/train_labels.csv")[:count * 100]
    #     dfY_merged = pd.merge(ypreddf.drop(["x", "y", "z"], axis=1), dfY, left_index=True, right_index=True)
    #     dfYidx = dfY.drop(["x", "y", "z", "qw", "qx", "qy", "qz"], axis=1)
    #     df_merged = pd.merge(ypreddf, dfYidx, left_index=True, right_index=True)
    #     df_merged["qw"] = 0
    #     df_merged["qx"] = 0
    #     df_merged["qy"] = 0
    #     df_merged["qz"] = 0
    #     df_merged.loc[df_merged["i"] == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
    #     df_merged.set_index(["chain_id", "i"], inplace=True)
    #    # df_merged.drop(["id"], axis=1, inplace=True)
    #     ToCSV(df_merged, "cnn-test")
    #     ToCSV(dfY_merged, "cnn-true")

    return model

# from https://github.com/1988kramer/camera-pose/blob/master/camera-pose.py 
def create_conv_branch(input_shape):
	model = Sequential()
	model.add(Conv2D(96, kernel_size=(11,11),
					 strides=4, padding='valid',
					 activation='relu',
					 input_shape=input_shape))
	model.add(MaxPooling2D(pool_size=(2,2), strides=2))
	# model.add(Conv2D(128, kernel_size=(5,5),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(MaxPooling2D(pool_size=(3,3), strides=1))
	# model.add(Conv2D(256, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(Conv2D(256, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(Conv2D(128, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(MaxPooling2D(pool_size=(3,3), strides=2))
	# model.add(Conv2D(256, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(Conv2D(128, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	return model

def create_conv_branch_pos(input_shape):
	model = Sequential()
	model.add(Conv2D(96, kernel_size=(11,11),
					 strides=4, padding='valid',
					 activation='relu',
					 input_shape=input_shape))
	model.add(MaxPooling2D(pool_size=(2,2), strides=2))
	model.add(Conv2D(128, kernel_size=(7,7),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(MaxPooling2D(pool_size=(3,3), strides=1))
	model.add(Conv2D(256, kernel_size=(5,5),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(256, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(128, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(MaxPooling2D(pool_size=(3,3), strides=2))
	model.add(Conv2D(256, kernel_size=(5,5),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(128, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	return model

In [20]:
# https://stackoverflow.com/questions/63827339/how-to-build-a-custom-data-generator-for-keras-tf-keras-where-x-images-are-being
class CustomDataGenerator(tf.keras.utils.Sequence):

    ''' Custom DataGenerator to load img 
    
    Arguments:
        data_frame = pandas data frame in filenames and labels format
        batch_size = divide data in batches
        shuffle = shuffle data before loading
        img_shape = image shape in (h, w, d) format
        augmentation = data augmentation to make model rebust to overfitting
    
    Output:
        Img: numpy array of image
        label : output label for image
    '''
    
    def __init__(self, data_frame, labels, batch_size=10, img_shape=None, augmentation=True, num_classes=None):
        self.data_frame = data_frame
        self.labels = labels
        self.train_len = len(data_frame)
        self.batch_size = batch_size
        self.img_shape = img_shape
        self.num_classes = num_classes
        print(f"Found {self.data_frame.shape[0]} images belonging to {self.num_classes} classes")

    def __len__(self):
        ''' return total number of batches '''
        #self.data_frame = shuffle(self.data_frame)
        return np.intmath.ceil(self.train_len/self.batch_size)

    def on_epoch_end(self):
        ''' shuffle data after every epoch '''
        # fix on epoch end it's not working, adding shuffle in len for alternative
        pass
    
    def __data_augmentation(self, img):
        ''' function for apply some data augmentation '''
        img = tf.image.resize(img, 512, 640)
        img = tf.image.random_flip_up_down(img)
        return img
        
    def __get_image(self, file_id):
        """ open image with file_id path and apply data augmentation """
        img = np.asarray(Image.open(file_id))
        img = np.resize(img, self.img_shape)
        img = self.__data_augmentation(img)

        return img

    def __get_label(self, label_id):
        """ uncomment the below line to convert label into categorical format """
        #label_id = tf.keras.utils.to_categorical(label_id, num_classes)
        return label_id

    def __getitem__(self, idx):
        # batch_x = self.data_frame["filenames"][idx * self.batch_size:(idx + 1) * self.batch_size]
        # batch_y = self.data_frame["labels"][idx * self.batch_size:(idx + 1) * self.batch_size]
        # # read your data here using the batch lists, batch_x and batch_y
        # x = [self.__get_image(file_id) for file_id in batch_x] 
        # y = [self.__get_label(label_id) for label_id in batch_y]

        y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        paths = self.data_frame["path"][idx * self.batch_size:(idx + 1) * self.batch_size]
        base_paths = self.data_frame["base_path"][idx * self.batch_size:(idx + 1) * self.batch_size]

        #raw = [tf.io.read_file(path) for path in paths]
        image = [tf.image.resize(tf.image.decode_png(tf.io.read_file(path), channels=3), (512,640)) for path in paths]
        #base_raw = tf.io.read_file(base_path)
        base_image = [tf.image.resize(tf.image.decode_png(tf.io.read_file(path), channels=3), (512,640)) for path in base_paths]

        print(y)
        return [np.asarray(base_image), np.asarray(image)], y

        #return tf.convert_to_tensor(x), tf.convert_to_tensor(y)

In [21]:



datagen = CustomDataGenerator(x_train, y_train, batch_size=100)

x, y = datagen[0]

print(y)


Found 1125 images belonging to None classes
               x           y           z        qw        qx        qy  \
807    27.300751  114.353508   31.839813  0.956902  0.198209  0.016586   
1229  601.903442   10.819830 -167.023010 -0.218553 -0.155113  0.821344   
1480   23.211967  -58.528553   47.175888 -0.923449  0.268661 -0.101654   
1436  115.176262   50.117092 -130.688095  0.736860 -0.248587 -0.486050   
179    88.315498  121.511284   35.013283  0.548183  0.582245 -0.306732   
...          ...         ...         ...       ...       ...       ...   
129   254.584839  -13.965038  -34.405235 -0.032231 -0.137024 -0.154052   
1061   79.531799  159.479446  -45.076118  0.899975  0.087413 -0.162958   
331   264.647552  154.280045  270.853485  0.235060  0.728259 -0.132490   
323   223.210693  249.846725  170.751617  0.652410  0.476736 -0.022427   
680   259.335876 -202.858765 -212.845001  0.682440  0.324751 -0.236397   

            qz  
807  -0.211606  
1229 -0.503556  
1480 -0.254420  

In [11]:
def NormalizeQuat(x: tf.Tensor):
    norm = tf.sqrt(tf.reduce_sum(tf.square(x)))
    x = x / norm
    return x

In [12]:

input_shape=(512, 640, 3)

# Rotation model
conv_branch = create_conv_branch(input_shape)
branch_a = Input(shape=input_shape)
branch_b = Input(shape=input_shape)

processed_a = conv_branch(branch_a)
processed_b = conv_branch(branch_b)

concat = keras.layers.concatenate([processed_a, processed_b])
x1 = Flatten()(concat)
x1 = Dropout(0.25)(x1)
x1 = Dense(8)(x1)
output = Dense(4)(x1)
postprocess = Lambda(NormalizeQuat, output_shape=(None,4))(output)

model_rot = Model(inputs=[branch_a,branch_b], outputs=postprocess)
model_rot.compile(optimizer=Adam(learning_rate=0.001), loss="mae")
model_rot.summary()
load_images_and_train_model(model_rot, False)

# Position model
# conv_branch = create_conv_branch_pos(input_shape)
# branch_a = Input(shape=input_shape)
# branch_b = Input(shape=input_shape)

# processed_a = conv_branch(branch_a)
# processed_b = conv_branch(branch_b)

# concat = keras.layers.concatenate([processed_a, processed_b])

# x1 = Flatten()(concat)
# x1 = Dropout(0.25)(x1)
# x1 = Dense(16)(x1)
# x1 = Dropout(0.25)(x1)
# x1 = Dense(16)(x1)
# x1 = Dropout(0.25)(x1)
# x1 = Dense(16)(x1)
# output = Dense(7)(x1)

# model_pos = Model(inputs=[branch_a,branch_b], outputs=output)
# model_pos.compile(optimizer=Adam(learning_rate=0.0001), loss="log_cosh")
# model_pos.summary()
# load_images_and_train_model(model_pos, True)

# yhat = model(x_test)
# var_inv = 1 / yhat.variance()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 512, 640,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 512, 640,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 63, 79,    │     34,944 │ input_layer_4[0]… │
│ (Sequential)        │ 96)               │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 63, 79,    │          0 │ sequential_1[0][… │
│ (Concatenate)       │ 192)              │            │ sequential_1[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 955584)    │          0 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 955584)    │          0 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8)         │  7,644,680 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 4)         │         36 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, None, 4)   │          0 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,679,660 (29.30 MB)

 Trainable params: 7,679,660 (29.30 MB)

...

 Total params: 7,679,660 (29.30 MB)

 Trainable params: 7,679,660 (29.30 MB)

 Non-trainable params: 0 (0.00 B)

Found 1125 images belonging to None classes


AttributeError: module 'numpy' has no attribute 'intmath'

In [ ]:
# Make predictions
ypred = pd.DataFrame(model_rot.predict(CustomDataGenerator(x_test, y_test, 20)), index=y_test.index, columns=["qw", "qx", "qy", "qz"]) # Predict rotation
#ypred = pd.DataFrame(model_pos.predict(CustomDataGenerator(x_test, y_test, 50)), index=y_test.index, columns=["x", "y", "z", "qw", "qx", "qy", "qz"]) # Predict position
#ypred = pd.merge(ypred_pos, ypred_rot, left_index=True, right_index=True)
#ypred[["qw", "qx", "qy", "qz"]] = ypred[["qw", "qx", "qy", "qz"]].clip(lower=-1, upper=1)

ypred["chain_id"] = dfY.iloc[ypred.index].chain_id
ypred["i"] = dfY.iloc[ypred.index].i.astype(int)
# y_pred_norm = np.linalg.norm(ypred[["qw", "qx", "qy", "qz"]], axis=1)
# ypred["qw"] = ypred["qw"] / y_pred_norm
# ypred["qx"] = ypred["qx"] / y_pred_norm
# ypred["qy"] = ypred["qy"] / y_pred_norm
# ypred["qz"] = ypred["qz"] / y_pred_norm
#ypred.loc[ypred.i == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
# ypred[["qw", "qx", "qy", "qz"]] = [0,0,0,0]
ypred[["x", "y", "z"]] = [0,0,0]
ypred = ypred[["chain_id", "i", "x", "y", "z", "qw", "qx", "qy", "qz"]]
ypred.loc[ypred.i == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
ypred.set_index(["chain_id", "i"], inplace=True)
ToCSV(ypred, "cnn-test")
y_copy = y_test.copy()
y_copy["chain_id"] = dfY.iloc[y_copy.index].chain_id
y_copy["i"] = dfY.iloc[y_copy.index].i.astype(int)
y_copy.set_index(["chain_id", "i"], inplace=True)
ToCSV(y_copy, "cnn-true")

#model.save('../models/test_model.keras')